# Darts LSTM model

In [ ]:
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts import metrics
from darts.dataprocessing.transformers import Scaler

import matplotlib.pyplot as plt

from src.resrnn import ResRNNModel

## Data preparation

In [ ]:
dset = pd.read_csv("../results/dataset_minutes.csv")
dset

In [ ]:
material = "G4"
sample = "S2"
train_ratio = 0.4

In [ ]:
def prepare_col(dset, col):
    serie = TimeSeries.from_dataframe(dset, time_col="Time [min]", value_cols=col)
    return serie.astype(np.float32)

In [ ]:
dset_sample = dset[(dset["material"] == material) & (dset["sample"] == sample)]
serie_wear = prepare_col(dset_sample, "Wear Loss [mm]")
serie_idx = prepare_col(dset_sample, "Time [min]")
serie_train, serie_test = serie_wear.split_after(train_ratio)

In [ ]:
_, ax = plt.subplots(figsize=(7, 4))
serie_train.plot(label="train", ax=ax)
serie_test.plot(label="test", ax=ax)
_ = ax.set_ylabel("Wear Loss [mm]")

In [ ]:
scaler = Scaler()
serie_train_scaled, serie_idx_scaled = scaler.fit_transform([serie_train, serie_idx])

## Model training

Here we define some key hyperparameters for the `RNNModel` class.
See [Darts documentation](https://unit8co.github.io/darts/generated_api/darts.models.forecasting.rnn_model.html) for more information.

In [ ]:
warmup = 250
model = ResRNNModel(
    model="LSTM",
    input_chunk_length=warmup,
    training_length=warmup,
    hidden_dim=64,
    n_rnn_layers=1,
    batch_size=32,
    dropout=0.0,
    n_epochs=100,
    optimizer_kwargs={"lr": 1e-3},
    random_state=42,
    force_reset=True,
    log_tensorboard=True,
    show_warnings=True,
    pl_trainer_kwargs={"accelerator": "gpu", "devices": 1},
)

In [ ]:
model.fit(serie_train_scaled, future_covariates=serie_idx_scaled)

## Predictions

In [ ]:
serie_forecast_scaled = model.predict(
    len(serie_test), future_covariates=serie_idx_scaled
)
serie_forecast = scaler.inverse_transform(serie_forecast_scaled)

In [ ]:
_, ax = plt.subplots(figsize=(7, 4))
serie_train.plot(label="train", ax=ax)
serie_test.plot(label="test", ax=ax)
serie_forecast.plot(label="predictions", ax=ax)
_ = ax.set_ylabel("Wear Loss [mm]")

We compute errors between obserbation and predictions but only at the **end point** of the trace.

In [ ]:
scores_functions = {
    "MAE": metrics.mae,
    "RMSE": metrics.rmse,
    "MAPE": metrics.mape,
}

In [ ]:
scores = {
    key: func(serie_test[-1], serie_forecast[-1])
    for key, func in scores_functions.items()
}
scores = (
    pd.DataFrame.from_dict(scores, orient="index", columns=("value",))
    .reset_index()
    .rename(columns={"index": "metric"})
)

In [ ]:
scores